In [ ]:
import torch
import sys
import gc
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, WrapperConfig, SMOL_LM_135M

In [41]:
cfg = WrapperConfig(
    model_id_or_path=SMOL_LM_135M,
    max_samples=1000,
    n_epochs=10,
    data_module_choice="ultra_feedback",
    notebook_mode=True,
    tuning_mode="full",
    eval_data_mode="fixed"
)

wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2024-12-18 17:04:24.642 | INFO     | trl_wrapper.trainer_wrapper:init_model:91 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct


In [42]:
wrapper.init_data_module()

2024-12-18 17:04:28.492 | INFO     | model.utils:setup:102 - Loading dataset for stage fit
2024-12-18 17:04:28.493 | INFO     | model.utils:setup:104 - Processing dataset for stage fit, workers: 1, cache dir dataset_caches/ultrafeedback
2024-12-18 17:04:29.698 | INFO     | dataset.squad:load_dataset:248 - Loaded dataset with 60917 samples
2024-12-18 17:04:29.783 | WARNING  | model.utils:setup:145 - Computed columns not found in dataset: ['source', 'prompt', 'chosen', 'chosen-rating', 'chosen-model', 'rejected', 'rejected-rating', 'rejected-model'] assuming using 


In [ ]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

In [43]:
wrapper.init_trainer()

2024-12-18 17:04:32.279 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:197 - Initializing DPOtrainer, run: run-256572, project: codecontests-llama-3b, logprobs cache: dataset_caches/ultrafeedback/b68b303b/ref_logprobs_cache peft config: False
2024-12-18 17:04:32.407 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:223 - Loading cached logprobs...
2024-12-18 17:04:32.568 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:229 - Loaded.


In [44]:
wrapper.train()

2024-12-18 17:04:34.210 | INFO     | trl_wrapper.trainer_wrapper:train:330 - Starting training.
2024-12-18 17:04:34.715 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:142 - Generating samples...
2024-12-18 17:04:34.715 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:60 - Generating policy samples, max length: 1024...
2024-12-18 17:04:39.764 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:78 - Generating reference samples, max length: 1024...
2024-12-18 17:04:59.818 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:208 - Saved eval samples.


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
0,No log,0.693147,0.000000,0.000000,0.000000,0.000000,-598.152710,-422.746918,3.293416,3.402780
100,0.693100,0.693147,0.000000,0.000000,0.000000,0.000000,-598.186035,-423.063049,3.297504,3.409995


Could not estimate the number of tokens of the input, floating-point operations will not be computed
2024-12-18 17:05:27.374 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:142 - Generating samples...
2024-12-18 17:05:27.374 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:60 - Generating policy samples, max length: 1024...
2024-12-18 17:05:32.424 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:78 - Generating reference samples, max length: 1024...
2024-12-18 17:05:52.508 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:208 - Saved eval samples.
2024-12-18 17:06:19.635 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:142 - Generating samples...
2024-12-18 17:06:19.636 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:60 - Generating policy samples, max length: 1024...
2024-12-18 17:06:24.688 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:78 - Generating reference samples, max length: 1024...


KeyboardInterrupt: 

In [ ]:
from IPython.display import Markdown
for prompt, chosen, rejected, logprob_diff in []:
    display(Markdown(f"\n### Prompt: {prompt}\n\n### Chosen:\n {chosen}\n\n### Rejected:\n {rejected}\n\nLogprob diff: {logprob_diff}"))